# Chopper Cascades

In [ ]:
%matplotlib widget
from scippneutron.tof import chopper_cascade
import scipp as sc

As an example, consider the WFM chopper cascade from [the tof package documentation](https://tof.readthedocs.io/en/stable/short-example.html):

In [ ]:
wfm1 = chopper_cascade.Chopper(
    distance=sc.scalar(6.6, unit='m'),
    time_open=sc.array(
        dims=('cutout',),
        values=[
            -0.008500,
            -0.006246,
            -0.004152,
            -0.002206,
            -0.000396,
            0.001286,
            0.005786,
            0.008039,
            0.010133,
            0.012080,
            0.013889,
            0.015571,
        ],
        unit='s',
    ),
    time_close=sc.array(
        dims=('cutout',),
        values=[
            -0.008063,
            -0.005639,
            -0.003387,
            -0.001292,
            0.000654,
            0.002464,
            0.006222,
            0.008646,
            0.010899,
            0.012993,
            0.014939,
            0.016750,
        ],
        unit='s',
    ),
)
wfm2 = chopper_cascade.Chopper(
    distance=sc.scalar(7.1, unit='m'),
    time_open=sc.array(
        dims=('cutout',),
        values=[
            -0.008063,
            -0.005640,
            -0.003387,
            -0.001292,
            0.000654,
            0.002451,
            0.006222,
            0.008645,
            0.010898,
            0.012993,
            0.014940,
            0.016737,
        ],
        unit='s',
    ),
    time_close=sc.array(
        dims=('cutout',),
        values=[
            -0.007627,
            -0.005033,
            -0.002621,
            -0.000527,
            0.001567,
            0.003641,
            0.006658,
            0.009252,
            0.011664,
            0.013759,
            0.015853,
            0.017927,
        ],
        unit='s',
    ),
)
frame_overlap_1 = chopper_cascade.Chopper(
    distance=sc.scalar(8.8, unit='m'),
    time_open=sc.array(
        dims=('cutout',),
        values=[
            -0.011062,
            -0.007837,
            -0.005124,
            -0.002594,
            -0.000139,
            0.002460,
            0.006796,
            0.010020,
            0.012733,
            0.015263,
            0.017718,
            0.020317,
        ],
        unit='s',
    ),
    time_close=sc.array(
        dims=('cutout',),
        values=[
            -0.010040,
            -0.006687,
            -0.004043,
            -0.001711,
            0.000640,
            0.003671,
            0.007817,
            0.011171,
            0.013814,
            0.016146,
            0.018497,
            0.021528,
        ],
        unit='s',
    ),
)
frame_overlap_2 = chopper_cascade.Chopper(
    distance=sc.scalar(15.9, unit='m'),
    time_open=sc.array(
        dims=('cutout',),
        values=[
            -0.024775,
            -0.019219,
            -0.013981,
            -0.009299,
            -0.004834,
            -0.000306,
            0.010939,
            0.016495,
            0.021733,
            0.026416,
            0.030880,
            0.035409,
        ],
        unit='s',
    ),
    time_close=sc.array(
        dims=('cutout',),
        values=[
            -0.021144,
            -0.015642,
            -0.010984,
            -0.006632,
            -0.002398,
            0.002582,
            0.014570,
            0.020072,
            0.024730,
            0.029082,
            0.033316,
            0.038297,
        ],
        unit='s',
    ),
)
pulse_overlap = chopper_cascade.Chopper(
    distance=sc.scalar(22.0, unit='m'),
    time_open=sc.array(dims=('cutout',), values=[-0.130952, 0.011905], unit='s'),
    time_close=sc.array(dims=('cutout',), values=[-0.087302, 0.055556], unit='s'),
)

In [ ]:
choppers = [wfm1, wfm2, frame_overlap_1, frame_overlap_2, pulse_overlap]

In [ ]:
frames = chopper_cascade.FrameSequence.from_source_pulse(
    time_min=sc.scalar(0.0, unit='ms'),
    time_max=sc.scalar(4.0, unit='ms'),
    wavelength_min=sc.scalar(0.01, unit='angstrom'),
    wavelength_max=sc.scalar(10.0, unit='angstrom'),
)
frames = frames.chop(choppers)
at_sample = frames.propagate_to(sc.scalar(26.0, unit='m'))

at_sample.draw()

We can also draw a chopper acceptance diagram, which is essentially the same as above, but propagated back to the source pulse distance:

In [ ]:
frames.acceptance_diagram()

## Frame unwrapping

For unwrapping frames, we need the bounds of the entire from, to determine times at which to cut.
Since $L_2$ can be different for every detector, this cutting time is different for every detector.
We can compute the frame bounds at a common distance, e.g., the sample, and propagate the bounds to the detectors:

In [ ]:
bounds = at_sample[-1].bounds()
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

For WFM, we need to compute subframe time cutting points.
Again, $L_2$ can be different for every detector, so we need to compute the cutting points for every detector:

In [ ]:
bounds = at_sample[-1].subbounds()
bounds

In [ ]:
chopper_cascade.propagate_times(
    time=bounds['time'],
    wavelength=bounds['wavelength'],
    distance=sc.linspace('L2', 1.0, 2.0, 100, unit='m'),
)

Note that subframes may in principle overlap, and this may depend on the detector.
We therefore call `subframe_bounds` at a common location and propagate the result, otherwise we would get a different number of subframes for different detectors.